# CURRENT

In [7]:
import time
start1 = time.time()
from demo2_utils import *
from crowning2_utils import *
from crown_utils import *
import time
print(int(time.time()-start1),'s')

def gen_crown1(zip_order_path, toothlib_scale=1):
    print(f'Run: gen_crown1')
    my_crown = None
    mesh_prep_fname, mesh_prep_3m = load_prep_from_zip(zip_order_path)
    mesh_big, mesh_small = get_two_largest_meshes_3m(mesh_prep_3m)
    my_case_details = get_toothnum(zip_order_path)
    tooth_filepath = os.path.join(tooth_dirpath_ottawa, f"{my_case_details[1]}.stl")
    tooth_mesh = pv.read(tooth_filepath)
    largest_plane = get_largest_axis_idx(mesh_prep_3m.bounds)
    if largest_plane in [0,2]:
        if largest_plane == 2:
            tooth_mesh1 = get_horizontal_position(mesh_prep_3m.bounds, mesh_small.centroid, tooth_mesh)
        elif largest_plane == 0:
            tooth_mesh1 = get_vertical_position(mesh_prep_3m.bounds, mesh_small.centroid, tooth_mesh)
        tooth_mesh2 = tooth_mesh1.scale(np.full(3, toothlib_scale))
        tooth_mesh3 = tooth_mesh2.translate(gen_move_coords(mesh_small.centroid, tooth_mesh2.center))
        my_crown = tooth_mesh3.boolean_difference(pv.wrap(mesh_small)).fill_holes(5)
    else:
        print('y-plane largest')
    return my_crown

def plot_crown(gen_crown, zip_order_path):
    mesh_prep_fname, mesh_prep_3m = load_prep_from_zip(zip_order_path)
    mesh_big, mesh_small = get_two_largest_meshes_3m(mesh_prep_3m)
    my_case_details = get_toothnum(zip_order_path)
    cam_pos = 'xy'
    my_shape = (1,2)
    p1_ = pv.Plotter(shape=my_shape)
    p1_.subplot(0,0) #1st plot
    p1_.add_mesh(mesh_small)
    p1_.add_mesh(mesh_big)
    p1_.add_mesh(my_crown, color='gold', opacity=.7)
    p1_.add_text(f'{my_case_details[0]}', position='lower_right', font_size=10)
    p1_.camera_position=cam_pos
    p1_.show_grid()
    p1_.subplot(0,1) #2nd plot
    p1_.add_mesh(my_crown, color='gold')
    p1_.add_text(f'{my_case_details[1]}', position='upper_right')
    p1_.add_text('antagonist', position='lower_right', font_size=10)
    p1_.camera_position=cam_pos
    p1_.show()

0 s


In [ ]:
# filter 30,31,32 tooth num cases
start = time.time()
target_path_list = []
tooth30_idx_ls = []
ct = 0
for i,case in enumerate(os.listdir(udx_dirpath)):
    ct += 1
    case_path = os.path.join(udx_dirpath, case)
    my_case_details = get_toothnum(case_path)
    if not my_case_details:
        print(f'Not designed: {case_path}')
        # os.remove(case_path) 
        continue
    # if my_case_details[1] in [30, 31,32]:
    if str(my_case_details[1]) == '30':
        tooth30_idx_ls.append(i)
        target_path_list.append(case_path)

print(f'Total cases: {len(os.listdir(udx_dirpath))}\nFiltered cases: {len(target_path_list)}')
print(f'Runtime: {int(time.time()-start)}s')
target_idx = -1
print(f'30: {tooth30_idx_ls}')

In [ ]:
#PROCESS
# target_idx += 1

start1 = time.time()
file_path = target_path_list[target_idx]
mesh_prep_fname, mesh_prep_3m = load_prep_from_zip(file_path)
mesh_anta_fname, mesh_anta_3m = load_anta_from_zip(file_path)
mesh_big, mesh_small = get_two_largest_meshes_3m(mesh_prep_3m)
my_case_details = get_toothnum(file_path)
tooth_filepath = os.path.join(tooth_dirpath_ottawa, f"{my_case_details[1]}.stl")
tooth_mesh = pv.read(tooth_filepath)
tooth_desi = get_designer_tooth(file_path)

# case when z plane is larger than x plane
largest_plane = get_largest_axis_idx(mesh_prep_3m.bounds)
print(f'largest plane: {largest_plane}')
if largest_plane in [0,2]:
    if largest_plane == 2:
        tooth_mesh1 = get_horizontal_position(mesh_prep_3m.bounds, mesh_small.centroid, tooth_mesh)
    elif largest_plane == 0:
        tooth_mesh1 = get_vertical_position(mesh_prep_3m.bounds, mesh_small.centroid, tooth_mesh)
    tooth_mesh2 = tooth_mesh1.scale(np.full(3, 1.20))
    tooth_mesh3 = tooth_mesh2.translate(gen_move_coords(mesh_small.centroid, tooth_mesh2.center))
    my_crown = tooth_mesh3.boolean_difference(pv.wrap(mesh_small))
   
    cam_pos = 'xy'
    my_shape = (1,4)
    p1_ = pv.Plotter(shape=my_shape)
        
    p1_.subplot(0,0) #1st plot
    p1_.add_mesh(mesh_small)
    p1_.add_mesh(mesh_big)
    p1_.add_mesh(tooth_mesh3, 
                 color='gold')
    p1_.add_text(f'{my_case_details[0]}', position='lower_right', font_size=10)
    p1_.camera_position=cam_pos
    p1_.show_grid()

    p1_.subplot(0,1) #2nd plot
    p1_.add_mesh(pv.wrap(mesh_anta_3m), opacity=.8)
    # p1_.add_mesh(tooth_mesh3.translate(gen_move_coords(tooth_mesh3.center-np.array([0,0,1]), tooth_mesh3.center)), color='w')
    p1_.add_mesh(tooth_mesh3, color='gold')
    p1_.add_text(f'{my_case_details[1]}', position='upper_right')
    p1_.add_text('antagonist', position='lower_right', font_size=10)
    p1_.camera_position=cam_pos
    
    p1_.subplot(0,2) #3rd plot
    # p1_.add_mesh(my_crown, color='gold', opacity=.9)
    # p1_.add_points(tooth_edges, color='red', line_width=10)
    p1_.add_mesh(pv.wrap(mesh_small))
    p1_.add_text('auto', position='lower_right', font_size=10)
    # p1_.enable_surface_point_picking(callback=callback2, show_point=True)
    p1_.show_grid()
    p1_.camera_position=cam_pos

    p1_.subplot(0,3) #4th plot
    p1_.add_mesh(pv.wrap(mesh_small))
    p1_.add_mesh(tooth_desi, color='w', opacity=.9)
    p1_.add_text('designer', position='lower_right', font_size=10)
    p1_.show_grid()
    p1_.camera_position=cam_pos

    p1_.show()
else:
    print('y-plane largest')
print(int(time.time()-start1), 's')

In [ ]:
#PROCESS v2
# target_idx += 1

start1 = time.time()
file_path = target_path_list[target_idx]
mesh_prep_fname, mesh_prep_3m = load_prep_from_zip(file_path)
mesh_anta_fname, mesh_anta_3m = load_anta_from_zip(file_path)
mesh_big, mesh_small = get_two_largest_meshes_3m(mesh_prep_3m)
my_case_details = get_toothnum(file_path)
tooth_filepath = os.path.join(tooth_dirpath_ottawa, f"{my_case_details[1]}.stl")
tooth_mesh = pv.read(tooth_filepath)
tooth_desi = get_designer_tooth(file_path)

# get margin
margin = pv.wrap(mesh_small).extract_feature_edges(7).connectivity().extract_largest()
base_bot = pv.wrap(mesh_small).extract_feature_edges(90)
if not margin:
    print('Margin error')
    
elif base_bot.center == margin.center:
    print('Unable to detect margin')
    
else:
    # case when z plane is larger than x plane
    largest_plane = get_largest_axis_idx(mesh_prep_3m.bounds)
    print(f'largest plane: {largest_plane}')
    if largest_plane in [0,2]:
        if largest_plane == 2:
            tooth_mesh1 = get_horizontal_position(mesh_prep_3m.bounds, mesh_small.centroid, tooth_mesh)
        elif largest_plane == 0:
            tooth_mesh1 = get_vertical_position(mesh_prep_3m.bounds, mesh_small.centroid, tooth_mesh)
        # tooth_mesh1 = tooth_mesh1.rotate_z(-compute_angle2(mesh_small))
        
        tooth_mesh2 = tooth_mesh1.scale(np.full(3, 1.00))
        tooth_mesh3 = tooth_mesh2.translate(gen_move_coords(mesh_small.centroid, tooth_mesh2.center))
        # tooth_mesh3 = tooth_mesh2.translate(gen_move_coords(margin.center, tooth_mesh2.center))
        my_crown = tooth_mesh3.boolean_difference(pv.wrap(mesh_small)).fill_holes(5)
       
        cam_pos = 'xy'
        my_shape = (1,2)
        p1_ = pv.Plotter(shape=my_shape)
            
        p1_.subplot(0,0) #1st plot
        p1_.add_mesh(mesh_small)
        p1_.add_mesh(mesh_big)
        p1_.add_mesh(my_crown, 
                     color='gold', opacity=.7)
        p1_.add_text(f'{my_case_details[0]}', position='lower_right', font_size=10)
        p1_.camera_position=cam_pos
        p1_.show_grid()
    
        p1_.subplot(0,1) #2nd plot
        # p1_.add_mesh(pv.wrap(mesh_anta_3m), opacity=.8)
        # p1_.add_mesh(tooth_mesh3.translate(gen_move_coords(tooth_mesh3.center-np.array([0,0,1]), tooth_mesh3.center)), color='w')
        p1_.add_mesh(my_crown, color='gold')
        p1_.add_text(f'{my_case_details[1]}', position='upper_right')
        p1_.add_text('antagonist', position='lower_right', font_size=10)
        p1_.camera_position=cam_pos
        p1_.show()
    else:
        print('y-plane largest')
    print(int(time.time()-start1), 's')

In [2]:
tooth30_idx_ls = [8, 16, 26, 31, 34, 39, 43, 44, 58, 69, 86, 92, 94, 108, 117, 148, 150, 154, 157, 158, 169, 170, 180, 181, 183, 186, 192, 196, 221, 225, 250, 266]
my_idx = 0
my_orderpath = os.path.join(udx_dirpath, os.listdir(udx_dirpath)[tooth30_idx_ls[my_idx]])

In [ ]:
my_idx += 1

In [8]:
my_crown = gen_crown1(my_orderpath, 1.1)
plot_crown(my_crown, my_orderpath)

Run: gen_crown1
LOWER


Widget(value="<iframe src='http://localhost:50888/index.html?ui=P_0x1042d1190_4&reconnect=auto' style='width: …

# WORKSPACE

In [ ]:
# import pyvista as pvorigin = poly.center
origin[-1] -= poly.length / 3.0
projected = poly.project_points_to_plane(origin=origin)

# Display the results
p = pv.Plotter()
p.add_mesh(poly)
p.add_mesh(projected)
p.show()
from pyvista import examples

poly = examples.load_random_hills()
poly.plot()

In [ ]:
origin = poly.center
origin[-1] -= poly.length / 3.0
projected = poly.project_points_to_plane(origin=origin)

# Display the results
p = pv.Plotter()
# p.add_mesh(poly)
p.add_mesh(projected)
p.show()

In [ ]:
values = spider_cage(x, y, z)
values

In [ ]:
type(grid)

In [ ]:
mesh = grid.contour([1], values, method='marching_cubes')
mesh.plot()

In [ ]:
dist = np.linalg.norm(mesh.points, axis=1)
mesh.plot(scalars=dist, smooth_shading=True, specular=1, cmap="plasma", show_scalar_bar=False)

In [ ]:
test_mesh = pv.wrap(mesh_small).extract_feature_edges(7).connectivity(largest=True)

In [ ]:
p1_ = pv.Plotter(shape=my_shape)
            
p1_.subplot(0,0) #1st plot
p1_.add_mesh(mesh_small)
p1_.add_mesh(mesh_big)
p1_.add_mesh(tooth_desi, 
             color='gold', opacity=.7)
p1_.add_text(f'{my_case_details[0]}', position='lower_right', font_size=10)
p1_.camera_position=cam_pos
p1_.show_grid()

p1_.subplot(0,1) #2nd plot
# p1_.add_mesh(pv.wrap(mesh_anta_3m), opacity=.8)
# p1_.add_mesh(tooth_mesh3.translate(gen_move_coords(tooth_mesh3.center-np.array([0,0,1]), tooth_mesh3.center)), color='w')
p1_.add_mesh(tooth_desi, color='gold')
p1_.add_text(f'{my_case_details[1]}', position='upper_right')
p1_.add_text('antagonist', position='lower_right', font_size=10)
p1_.camera_position=cam_pos
p1_.show()

In [ ]:
mesh_small.volume

In [ ]:
tooth_desi.volume

In [ ]:
my_crown.volume

# CASE VIEWER PTS

In [ ]:
import json

tooth_num = 4
txt_path = '/Users/nimbyx/Downloads/2023-09-07T02_21_39.005Z.txt'
with open(txt_path, 'r') as f:
    my_pts = json.load(f)

pt1 = string_to_numpy_array(my_pts['points'][0]['firstPoint'])
pt2 = string_to_numpy_array(my_pts['points'][0]['secondPoint'])

tooth_filepath = os.path.join(tooth_dirpath_ottawa, f"{tooth_num}.stl")
tooth4 = pv.read(tooth_filepath)

p3 = pv.Plotter()
p3.add_points(np.array(pt1), color='r', point_size=15, render_points_as_spheres=True)
p3.add_points(np.array(pt2), color='blue', point_size=15, render_points_as_spheres=True)
p3.add_mesh(tooth4)

p3.show_grid()
p3.show()